# COVID-19 by U.S. County

This project arose out of an interest to visualize how COVID-19 hotspots have shifted over the course of the unfolding pandemic. The first U.S. cases started popping up in late January along the West Coast and New York, where people frequently travel to and from China, but in the time since, it has peaked in California, then New York, now (as of July 3, 2020, when this data was produced) the number of daily cases is rising rapidly in Florida, Texas, and California (again).

**🚨PUBLIC HEALTH ADVISORY 🚨**: Stay away from crowded beaches, and go make a friend with a backyard pool instead 😉 (as long as they've tested negative of course...and they don't invite other friends...you know what, just take a relaxing bath instead)

--

Ever since watching this video (https://www.youtube.com/watch?v=8WVoJ6JNLO8) about how the top 10 most valuable companies changed over the last two decades, I've been interested in time-based animations like the "racing bar chart"  shown in the video. Since COVID outbreaks are highly linked with geography, I'm going to attempt to recreate an animated map version to see what we can learn.

Along the way, you'll learn how to blend datasets with Python and create time-based animations with Tableau Public.

I will be using two datasets here - the first is from the New York Times showing COVID-19 cases by U.S. county  (https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv) (accurate up to July 2, 2020), and the second is a [table of population data for each U.S. county] (https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage).

A note on the NYT dataset: COVID cases are listed here as cumulative totals up to a given date, including asymptomatic  cases, symptomatic cases, and those who have already recovered. In other words, the number of cases may be an overestimate of how many people ACTUALLY had COVID at that given point in time. 

### Data Upload and Cleaning

The Kaggle dataset doesn't include information about county population, so I'm going to merge the two datasets into one using merge in Pandas. If you're not into Python, you could also accomplish this by using a SQL JOIN or a VLOOKUP in Excel. Before we do that, we first need to clean the data to get it in the most desirable format for merging.

In [2]:
import numpy as np
import pandas as pd

covid = pd.read_csv('covid-us-counties.csv')
covid.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [61]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288683 entries, 0 to 288682
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    288683 non-null  object 
 1   county  288683 non-null  object 
 2   state   288683 non-null  object 
 3   fips    285660 non-null  float64
 4   cases   288683 non-null  int64  
 5   deaths  288683 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 13.2+ MB


In [3]:
cpop = pd.read_excel('co-est2019-annres.xlsx',skiprows=2)

In [142]:
cpop.head()

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,Population Estimate 2019
0,".Autauga County, Alabama",54571.0,54597.0,54773.0,55227.0,54954.0,54727.0,54893.0,54864.0,55243.0,55390.0,55533.0,55869.0
1,".Baldwin County, Alabama",182265.0,182265.0,183112.0,186558.0,190145.0,194885.0,199183.0,202939.0,207601.0,212521.0,217855.0,223234.0
2,".Barbour County, Alabama",27457.0,27455.0,27327.0,27341.0,27169.0,26937.0,26755.0,26283.0,25806.0,25157.0,24872.0,24686.0
3,".Bibb County, Alabama",22915.0,22915.0,22870.0,22745.0,22667.0,22521.0,22553.0,22566.0,22586.0,22550.0,22367.0,22394.0
4,".Blount County, Alabama",57322.0,57322.0,57376.0,57560.0,57580.0,57619.0,57526.0,57526.0,57494.0,57787.0,57771.0,57826.0


The only two columns we're really interested in are the Geographic Area with relevant county information, and the population estimate for 2019. Let's grab those and ignore the rest.

In [4]:
# ignore all the explanations of the dataset at the very bottom
cpop = cpop[['Geographic Area','Population Estimate 2019']][0:3142]
cpop.tail(10)

,Geographic Area,Population Estimate 2019
3132,".Niobrara County, Wyoming",2356.0
3133,".Park County, Wyoming",29194.0
3134,".Platte County, Wyoming",8393.0
3135,".Sheridan County, Wyoming",30485.0
3136,".Sublette County, Wyoming",9831.0
3137,".Sweetwater County, Wyoming",42343.0
3138,".Teton County, Wyoming",23464.0
3139,".Uinta County, Wyoming",20226.0
3140,".Washakie County, Wyoming",7805.0
3141,".Weston County, Wyoming",6927.0


In [5]:
cpop['County'] = cpop['Geographic Area'].apply(lambda x: x.split(', ')[0][1:])
cpop['County'] = cpop['County'].apply(lambda x: x.split(' County')[0]) # for everywhere except AK + LA
cpop['County'] = cpop['County'].apply(lambda x: x.split(' Parish')[0]) # for LA
cpop['County'] = cpop['County'].apply(lambda x: x.split(' City')[0]) # for VA
cpop['State'] = cpop['Geographic Area'].apply(lambda x: x.split(', ')[1])
cpop.head(70)

,Geographic Area,Population Estimate 2019,County,State
0,".Autauga County, Alabama",55869.0,Autauga,Alabama
1,".Baldwin County, Alabama",223234.0,Baldwin,Alabama
2,".Barbour County, Alabama",24686.0,Barbour,Alabama
3,".Bibb County, Alabama",22394.0,Bibb,Alabama
4,".Blount County, Alabama",57826.0,Blount,Alabama
...,...,...,...,...
65,".Wilcox County, Alabama",10373.0,Wilcox,Alabama
66,".Winston County, Alabama",23629.0,Winston,Alabama
67,".Aleutians East Borough, Alaska",3337.0,Aleutians East Borough,Alaska
68,".Aleutians West Census Area, Alaska",5634.0,Aleutians West Census Area,Alaska


In [184]:
# Test to see if Alaska works now
cpop[cpop['State']=='Alaska']
# Two counties that both start with "Aleutians" have been combined, don't want to bother fixing that.

,Geographic Area,Population Estimate 2019,County,State
67,".Aleutians East Borough, Alaska",3337.0,Aleutians East Borough,Alaska
68,".Aleutians West Census Area, Alaska",5634.0,Aleutians West Census Area,Alaska
69,".Anchorage Municipality, Alaska",288000.0,Anchorage Municipality,Alaska
70,".Bethel Census Area, Alaska",18386.0,Bethel Census Area,Alaska
71,".Bristol Bay Borough, Alaska",836.0,Bristol Bay Borough,Alaska
72,".Denali Borough, Alaska",2097.0,Denali Borough,Alaska
73,".Dillingham Census Area, Alaska",4916.0,Dillingham Census Area,Alaska
74,".Fairbanks North Star Borough, Alaska",96849.0,Fairbanks North Star Borough,Alaska
75,".Haines Borough, Alaska",2530.0,Haines Borough,Alaska
76,".Hoonah-Angoon Census Area, Alaska",2148.0,Hoonah-Angoon Census Area,Alaska


In [185]:
# Test to see if Louisiana works now
cpop[cpop['State']=='Louisiana']
# 3 counties that all start with "West" have been combined, don't want to bother fixing that.

,Geographic Area,Population Estimate 2019,County,State
1113,".Acadia Parish, Louisiana",62045.0,Acadia,Louisiana
1114,".Allen Parish, Louisiana",25627.0,Allen,Louisiana
1115,".Ascension Parish, Louisiana",126604.0,Ascension,Louisiana
1116,".Assumption Parish, Louisiana",21891.0,Assumption,Louisiana
1117,".Avoyelles Parish, Louisiana",40144.0,Avoyelles,Louisiana
...,...,...,...,...
1172,".Webster Parish, Louisiana",38340.0,Webster,Louisiana
1173,".West Baton Rouge Parish, Louisiana",26465.0,West Baton Rouge,Louisiana
1174,".West Carroll Parish, Louisiana",10830.0,West Carroll,Louisiana
1175,".West Feliciana Parish, Louisiana",15568.0,West Feliciana,Louisiana


Alaska and Louisiana are good enough, what about New York City?

In [6]:
#covid[covid['county']=='New York City']['county'] = 'New York'
#covid[covid['county']=='New York City']['county'].apply(lambda x: x.split(' '))
covid.loc[covid['county'] == 'New York City','county'] = 'New York'
covid[covid['state']=='New York']

,date,county,state,fips,cases,deaths
416,2020-03-01,New York,New York,NaN,1,0
448,2020-03-02,New York,New York,NaN,1,0
482,2020-03-03,New York,New York,NaN,2,0
518,2020-03-04,New York,New York,NaN,2,0
519,2020-03-04,Westchester,New York,36119.0,9,0
...,...,...,...,...,...,...
287446,2020-07-01,Washington,New York,36115.0,246,11
287447,2020-07-01,Wayne,New York,36117.0,174,1
287448,2020-07-01,Westchester,New York,36119.0,34865,1558
287449,2020-07-01,Wyoming,New York,36121.0,95,3


In [7]:
df = pd.merge(covid,cpop,left_on=['county','state'],right_on=['County','State'],how='inner')
df

,date,county,state,fips,cases,deaths,Geographic Area,Population Estimate 2019,County,State
0,2020-01-21,Snohomish,Washington,53061.0,1,0,".Snohomish County, Washington",822083.0,Snohomish,Washington
1,2020-01-22,Snohomish,Washington,53061.0,1,0,".Snohomish County, Washington",822083.0,Snohomish,Washington
2,2020-01-23,Snohomish,Washington,53061.0,1,0,".Snohomish County, Washington",822083.0,Snohomish,Washington
3,2020-01-24,Snohomish,Washington,53061.0,1,0,".Snohomish County, Washington",822083.0,Snohomish,Washington
4,2020-01-25,Snohomish,Washington,53061.0,1,0,".Snohomish County, Washington",822083.0,Snohomish,Washington
...,...,...,...,...,...,...,...,...,...,...
285100,2020-06-30,Oneida,Idaho,16071.0,2,0,".Oneida County, Idaho",4531.0,Oneida,Idaho
285101,2020-07-01,Oneida,Idaho,16071.0,3,0,".Oneida County, Idaho",4531.0,Oneida,Idaho
285102,2020-07-01,Robertson,Kentucky,21201.0,1,0,".Robertson County, Kentucky",2108.0,Robertson,Kentucky
285103,2020-07-01,Hooker,Nebraska,31091.0,1,0,".Hooker County, Nebraska",682.0,Hooker,Nebraska


Note that we've done an inner merge, meaning that we'll only keep the county data if it's present in both the COVID reports and the census dataset. We don't need all these columns anymore, so let's create a new dataframe.

In [212]:
df.columns

Index(['date', 'county', 'state', 'fips', 'cases', 'deaths', 'Geographic Area',
       'Population Estimate 2019', 'County', 'State'],
      dtype='object')

In [28]:
dfm = df[['date','County','State','cases','deaths','Population Estimate 2019']]
dfm.tail(10)

,date,County,State,cases,deaths,Population Estimate 2019
285095,2020-06-29,Rush,Kansas,1,0,3036.0
285096,2020-06-30,Rush,Kansas,1,0,3036.0
285097,2020-07-01,Rush,Kansas,1,0,3036.0
285098,2020-06-30,Bear Lake,Idaho,1,0,6125.0
285099,2020-07-01,Bear Lake,Idaho,2,0,6125.0
285100,2020-06-30,Oneida,Idaho,2,0,4531.0
285101,2020-07-01,Oneida,Idaho,3,0,4531.0
285102,2020-07-01,Robertson,Kentucky,1,0,2108.0
285103,2020-07-01,Hooker,Nebraska,1,0,682.0
285104,2020-07-01,Harmon,Oklahoma,1,0,2653.0


In [214]:
dfm.to_excel('covid-us-county-population.xlsx')

## Cleaning Data (EVERYTHING IN THIS SECTION IS RESOLVED NOW)

(AK and LA data are missing, as is NYC)

They are present in the original data set, so they must have been eliminated when I performed the merge.

Here we can start to see that there's not a regularity with county names in Alaska. Let's see if it's the same in Louisiana. If it is, we can clean their rows, delete the old rows, and union the clean rows back into the data set.

(For alaska, split County by whitespace, then pick the -1'th element

## Figuring out proper gradations for the Tableau color maps

Let's split the colormap into 5 gradations based on quintiles. Quintiles are the same idea as percentiles, except instead of breaking up the data into 100 bins, we'll break them into 5. This means that the top quintile encompasses the top 20% of cases, or the 80th-99th percentiles. Make sense?


Let's calculate the 80th percentile for cases, deaths, and % of population infected. 

In [9]:
dfm.head()

,date,County,State,cases,deaths,Population Estimate 2019
0,2020-01-21,Snohomish,Washington,1,0,822083.0
1,2020-01-22,Snohomish,Washington,1,0,822083.0
2,2020-01-23,Snohomish,Washington,1,0,822083.0
3,2020-01-24,Snohomish,Washington,1,0,822083.0
4,2020-01-25,Snohomish,Washington,1,0,822083.0


In [18]:
dfdate = dfm[dfm['date']=='2020-07-01']
np.percentile(dfdate['cases'],80)

513.2000000000016

In [21]:
np.percentile(dfdate['deaths'],80)

15.0

Therefore, I'll edit the colorbars so that the last quintiles in the color gradient are 513 and 15 for the cases and deaths worksheets respectively.

## Areas for Improvement

#### Changing the fixed x-axis to a dynamic axis

Right now, the x-axis are fixed for all of the racing bar charts, whereas in the classic racing bar chart, the #1 ranked bar takes up the full-width of the screen, with every other bar taking up a width proportional to the #1 ranked  bar. This means that the resolution is too low to see what's really happening before, say, mid-April, making the animation pretty boring at first. There's a tutorial for how to fix this here (https://gravyanecdote.com/tableau/how-to-build-a-bar-chart-race-in-tableau/) which works for non-cumulative data, but since our raw data consists of the cumulative number of cases and deaths already, their fix with WINDOW_MAX doesn't work for mine.

The same principle could be applied to maps, but I decided not to because it could be deceptive for the viewer. Consider that Snohomish County, WA in late January had 1 case, with no cases anywhere else in the US. Since it's the 100th percentile, the county would be coded to dark red. Fast forward to July 1, where LA and NY counties are also coded as dark red for their relatively severe outbreaks of 100k+ cases. 1 case in WA is incomparable to 100k cases in LA county, so it doesn't make sense for them to both be coded as dark red. 

#### Creating a live connection

I downloaded this dataset from NYT's Github page on July 1, 2020, and quite a lot of cases and new trends have emerged since then that have not been captured with these animations. At the moment, I'm using Tableau Public (Tableau's free offering), which doesn't allow for live connections with datasets, so if I upgraded to the full version of Tableau Desktop, I could set up a live connection with NYT's Github page so that the data updates automatically every day, and the visualizations update to match.

#### Speed

This dataset has ~280k rows, so every time you zoom in or out on the map, the whole map needs to be redrawn, making the interface very laggy at times. I don't know if this is fixable with Tableau, but I'd love to learn if anyone else knows how...